In [1]:
#Grid de la segunda parte, en la que utilizamos ya la primera parte bien
#Esta va a ser la de las 44 variables con combined


#importamos paquetes
import sys
sys.path.append('Src/')
from data_modificado import * #hay funciones que estan cambiadas en este script para adaptralas a nuestro dataset
from train_2 import * #este hubo que modificar una linea tambien
from transfer_learning import * #hubo que modificart lo mismo que en train_2
from test_functions import *
from layers import *
from utils import *
from loss import *
from metric import *
from results import *
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
import itertools as it

#funciones
def read_df(
              metadata_names=['age','Temperature','Precipitation3Days'],
              random_state=42,
              otu_filename='../Datasets/otu_table_all_80.csv',
              metadata_filename='../Datasets/metadata_table_all_80.csv'):
    otu = pd.read_csv(otu_filename, index_col=0, header=None).T
    #print(otu.head())
    otu = otu.set_index('otuids')
    otu = otu.astype('int32')
    metadata = pd.read_csv(metadata_filename)
    #print(metadata.head())
    metadata = metadata.set_index('X.SampleID')
    metadata.head()
    domain = metadata[metadata_names]
    #if 'INBREDS' in metadata_names:
    #    domain = pd.concat([domain, pd.get_dummies(domain['INBREDS'], prefix='INBREDS')], axis=1)
    #    domain = domain.drop(['INBREDS'], axis=1)
    #elif 'Maize_Line' in metadata_names:
    #    domain = pd.concat([domain, pd.get_dummies(domain['Maize_Line'], prefix='Maize_Line')], axis=1)
    #    domain = domain.drop(['Maize_Line'], axis=1) 
    df = pd.concat([otu, domain], axis=1, sort=True, join='outer')
    #print(df.head())
    #data_microbioma = df[otu.columns].to_numpy(dtype=np.float32)
    #data_domain = df[domain.columns].to_numpy(dtype=np.float32)
    df_microbioma = df[otu.columns]
    df_domain = df[domain.columns]
    df_domain.head()
    df_microbioma_train, df_microbioma_no_train, df_domain_train, df_domain_no_train = \
        train_test_split(df_microbioma, df_domain, test_size=0.1, random_state=random_state)
    # Transfer learning subset
    df_microbioma_test, df_microbioma_transfer_learning, df_domain_test, df_domain_transfer_learning = \
        train_test_split(df_microbioma_no_train, df_domain_no_train, test_size=0.1, random_state=random_state)
    df_microbioma_transfer_learning_train, df_microbioma_transfer_learning_test, df_domain_transfer_learning_train, df_domain_transfer_learning_test = \
        train_test_split(df_microbioma_transfer_learning, df_domain_transfer_learning, test_size=0.3, random_state=random_state)
    
    return df_microbioma_train, df_microbioma_test, df_microbioma_transfer_learning_train, df_microbioma_transfer_learning_test, df_domain_train, df_domain_test, df_domain_transfer_learning_train, df_domain_transfer_learning_test, otu.columns, domain.columns
    #return df_microbioma_train, df_microbioma_test, df_microbioma_transfer_learning_train, df_microbioma_transfer_learning_test, df_domain_train, df_domain_test, df_domain_transfer_learning_train, df_domain_transfer_learning_test, otu.columns, domain.columns


def train_kfold_mod(model_fn, m_train, d_train, z_train, m_test, d_test, z_test,
                batch_size, epochs, train_callbacks):
    all_models = model_fn()
    model, encoder_bioma, encoder_domain, decoder_bioma = all_models
    metrics_prefix = None
    if encoder_bioma is not None and encoder_domain is not None:
        x_train = (m_train, d_train)
        y_train = (m_train, m_train, z_train)
        x_test = (m_test, d_test)
        y_test = (m_test, m_test, z_test)
    elif encoder_bioma is not None:
        x_train = m_train
        y_train = m_train
        x_test = m_test
        y_test = m_test
        metrics_prefix = 'bioma'
    elif encoder_domain is not None:
        x_train = d_train
        y_train = m_train
        x_test = d_test
        y_test = m_test
        metrics_prefix = 'domain'

    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(5000).batch(
        batch_size)
    train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)
    val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)
    val_dataset = val_dataset.prefetch(tf.data.experimental.AUTOTUNE)
    r = model.fit(train_dataset,
                  epochs=epochs,
                  validation_data=val_dataset,
                  callbacks=train_callbacks,
                  verbose=0)
    if metrics_prefix is not None:
        old_keys = r.history
        r.history = {}
        for k, v in old_keys.items():
            if k == 'loss' or k == 'val_loss':
                new_key = k
            elif k.startswith('val_'):
                new_key = 'val_{}_{}'.format(metrics_prefix, k[4:])
            else:
                new_key = '{}_{}'.format(metrics_prefix, k)
            r.history[new_key] = v
    del val_dataset
    del train_dataset
    del x_train
    del y_train
    del x_test
    del y_test
    return r, all_models

def train_2(model_fn,
          data_microbioma,
          data_domain,
          latent_space=10,
          folds=5,
          epochs=20,
          batch_size=128,
          learning_rate_scheduler=ExpDecayScheluder(),
          random_seed=347,
          verbose=0):
    data_zeros_latent = np.zeros((data_microbioma.shape[0], latent_space), dtype=data_microbioma.dtype)
    results = []
    models = []
    train_callbacks = [
        callbacks.EarlyStopping(monitor='val_loss', patience=epochs + 1, restore_best_weights=True)]
    if verbose >= 0:
        train_callbacks += [TqdmCallback(verbose=verbose)]
    if learning_rate_scheduler is not None:
        train_callbacks += [learning_rate_scheduler.make()]

    if folds <= 1:
        m_train, m_test = data_microbioma, data_microbioma
        d_train, d_test = data_domain, data_domain
        z_train, z_test = data_zeros_latent, data_zeros_latent
        tf.random.set_seed(random_seed)
        r, m = train_kfold(model_fn, m_train, d_train, z_train, m_test, d_test, z_test,
                           batch_size, epochs, train_callbacks)
        results.append(r)
        models.append(m)

    else: #EL PROBLEMA ESTA AQUI, QUE HACE FALTA UN 
        kf = KFold(n_splits=folds, random_state=random_seed, shuffle=True)
        tf.random.set_seed(random_seed)

        for train_index, test_index in kf.split(data_microbioma):
            m_train, m_test = data_microbioma[train_index], data_microbioma[test_index]
            #print(m_train)
            #d_train, d_test = data_domain[train_index], data_domain[test_index]
            if data_domain is None:
                d_train, d_test = None, None
            else:
                d_train, d_test = data_domain[train_index], data_domain[test_index]
            #print(d_train)
            #Esto de hacer el if else ha funcionado, pero no se si hace lo que debe bien
            z_train, z_test = data_zeros_latent[train_index], data_zeros_latent[test_index]
            r, m = train_kfold_mod(model_fn, m_train, d_train, z_train, m_test, d_test, z_test,
                               batch_size, epochs, train_callbacks)
            results.append(r)
            models.append(m)
    return results, models

def perform_experiment_2_mod(cv_folds, epochs, batch_size, learning_rate, optimizer,
                       learning_rate_scheduler, input_transform, output_transform,
                       reconstruction_loss, latent_space, layers,
                       activation, activation_latent,
                       data_microbioma_train, data_domain_train,
                       show_results=True, device='/CPU:0'): #Show results cambiado de False  aTrue
    if input_transform is not None:
        input_transform = input_transform()
    #----------    
    if output_transform is not None:
        output_transform = output_transform()
    #----------      
    if reconstruction_loss.__class__.__name__ == 'MakeLoss':
        reconstruction_loss = reconstruction_loss.make()
    else:
        reconstruction_loss = reconstruction_loss()
    domain_layers = [l // 16 for l in layers] ####que es esto???? Esto es para las capas del domain
    #print(domain_layers)
    bioma_autoencoder = " -> ".join(["b"] +
                                    [str(l) for l in layers] +
                                    [str(latent_space)] +
                                    [str(l) for l in reversed(layers)] +
                                    ["b"])
    #---------- 
    #esto solo se utiliza para el texto, es irrelevante para nuestro error
    if data_domain_train is not None:
        domain_autoencoder = " -> ".join(["d"] +
                                     [str(l) for l in domain_layers] +
                                     [str(latent_space)] +
                                     [str(l) for l in reversed(layers)] +
                                     ["b"])
        
    else: 
        domain_autoencoder = " "
    #---------- 
    #donde se usa domain autoencoder?
    in_transform_name = input_transform.__class__.__name__ if input_transform else "none"
    out_transform_name = output_transform.__class__.__name__ if output_transform else "none"
    lr_scheduler_text = learning_rate_scheduler[
        1] if learning_rate_scheduler is not None else "none"
    lr_text = learning_rate if learning_rate_scheduler is not None else "constant = {}".format(
        learning_rate)
    learning_rate_scheduler = learning_rate_scheduler[
        0] if learning_rate_scheduler is not None else None
    optimizer = optimizer(learning_rate=learning_rate)
    #---------- 
    experiment_parameters = [
        ("Input transform", in_transform_name),
        ("Output transform", out_transform_name),
        ("Reconstruction Loss", reconstruction_loss.__class__.__name__),
        ("Latent Space", latent_space),
        ("Bioma Autoencoder", bioma_autoencoder),
        ("Domain Autoencoder", domain_autoencoder),
        ("Activation Encoder", activation),
        ("Activation Decoder", activation),
        ("Activation Latent", activation_latent),
        ("CV folds", cv_folds),
        ("Epochs", epochs),
        ("Batch Size", batch_size),
        ("Learning Rate Scheduler", lr_scheduler_text),
        ("Learning Rate", lr_text),
        ("Optimizer", optimizer.__class__.__name__),
    ]
    #----------  
    if show_results:
        md_text = ""
        md_text += "| Parameter             | Value         |\n"
        md_text += "|:----------------------|:--------------|\n"
        for n, v in experiment_parameters:
            md_text += "| {} | {} |\n".format(n, v)

        display(Markdown(md_text))
    #------------
    def create_model(print_data=False):
        bioma_shape=data_microbioma_train.shape[1]
        
        if data_domain_train is not None:
            domain_shape=data_domain_train.shape[1]
            #print("data_domain_train!=None")
        else:
            domain_shape=None
            #print("data_domain_train==None")
        models = autoencoder(bioma_shape=bioma_shape,
                             #bioma_shape=717,
                             domain_shape=domain_shape,
                             output_shape=bioma_shape,
                             #output_shape=717,
                             latent_space=latent_space,
                             bioma_layers=layers, #Esto es lo de [512,316]
                             domain_layers=domain_layers, #Esto son cada una de las layers divididas por 16
                             input_transform=input_transform,
                             output_transform=output_transform,
                             activation_function_encoder=activation,
                             activation_function_decoder=activation,
                             activation_function_latent=activation_latent)
        #Entiendo analizando lo demas que aqui NO esta el error
        #la funcion autoencoder esta en model.py (es la unica funcion en ese script)
        
        model, encoder_bioma, encoder_domain, decoder_bioma = models

        if print_data:
            plot_models(model, encoder_bioma, encoder_domain, decoder_bioma)
        compile_train(model,
                      encoder_bioma=encoder_bioma,
                      encoder_domain=encoder_domain,
                      reconstruction_error=reconstruction_loss,
                      encoded_comparison_error=losses.MeanAbsoluteError(),
                      metrics=get_experiment_metrics(input_transform, output_transform),
                      optimizer=optimizer)
        
        #print("He acabado create_model :)")
        return model, encoder_bioma, encoder_domain, decoder_bioma
    #-----------
    create_model(print_data=False)
    #-----------
    #Esta en esta seccion el problema, en train_2
    #print(data_domain_train)
    #print(latent_space)
    with tf.device(device):
        results, models = train_2(create_model,
                                data_microbioma_train,
                                data_domain_train,
                                latent_space=latent_space,
                                folds=cv_folds,
                                epochs=epochs,
                                batch_size=batch_size,
                                learning_rate_scheduler=learning_rate_scheduler,
                                verbose=-1)
    #----------
    validation_results = print_results(results, show_results=show_results)
    if show_results:
        display(Markdown("*************"))

    return experiment_parameters + validation_results, models, results

__params:__
 - "activat_func":["softmax","sigmoid","relu","tanh"]
 - "activ_ouput":["softmax","sigmoid","relu","tanh"]
 - "learning_rate":[0.01,0.001]

In [2]:
#Cargamos los datos
#nombres_metadatos = ["KCAL","PROT","TFAT","CARB","MOIS","ALC","CAFF","THEO","SUGR","FIBE","CALC","IRON","MAGN","PHOS","POTA","SODI","ZINC","COPP","SELE","VC","VB1","VB2","NIAC","VB6","FOLA","VB12","VARA","RET","BCAR","ACAR","CRYP","LYCO","LZ","ATOC","VK","CHOLE","SFAT","MFAT","PFAT","VITD","CHOLN"]
nombres_metadatos = ["PROT","TFAT","CARB","MOIS","ALC","CAFF","THEO","CALC","MAGN","POTA","ZINC","VC","VB1","VB6","VARA","ACAR","CRYP","LYCO","ATOC","VK","CHOLE","VITD"]
df_microbioma_train, df_microbioma_test, _, _, \
df_domain_train, df_domain_test, _, _, otu_columns, domain_columns = read_df(metadata_names=nombres_metadatos,otu_filename='resultados_ana/datos_otus_filtrados/otu_table_especies_80.csv',metadata_filename='resultados_ana/metadatos_nutrientes_filtrados_22.csv')

data_microbioma_train = df_microbioma_train.to_numpy(dtype=np.float32)
data_microbioma_test = df_microbioma_test.to_numpy(dtype=np.float32)
data_domain_train = df_domain_train.to_numpy(dtype=np.float32)
data_domain_test = df_domain_test.to_numpy(dtype=np.float32)

#Preparamos las combinaciones pertinentes (5 mejores) --> ####provisionales
combinations = [[100,96,0.01,optimizers.Adam,15,[512,256],"tanh","tanh"],\
                [100,64,0.01,optimizers.Adam,15,[512,256],"tanh","tanh"],\
                [100,64,0.001,optimizers.Adam,15,[512,256,128],"tanh","tanh"],\
                [100,64,0.001,optimizers.Adam,15,[512,256],"tanh","tanh"],\
                [100,64,0.01,optimizers.Adam,15,[512,256],"tanh","softmax"]]

### 0.001, tanh, tanh

In [3]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='tanh')(in_layer)
        net = layers.Dense(64, activation='tanh')(net)
        net = layers.Dense(32, activation='tanh')(net)
        net = layers.Dense(16, activation='tanh')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation=None)(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6897348165512085 | 0.6897348165512085 | 0.6897348165512085 |
| pearson_corr | 0.5599374771118164 | 0.5599374771118164 | 0.5599374771118164 |
| jensen_shannon_divergence | 10.814908027648926 | 10.814908027648926 | 10.814908027648926 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4639817178249359 | 0.4639817178249359 | 0.4639817178249359 |
| pearson_corr | 0.636695384979248 | 0.636695384979248 | 0.636695384979248 |
| jensen_shannon_divergence | 1.289400339126587 | 1.289400339126587 | 1.289400339126587 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.45759910345077515 | 0.45759910345077515 | 0.45759910345077515 |
| pearson_corr | 0.6679344773292542 | 0.6679344773292542 | 0.6679344773292542 |
| jensen_shannon_divergence | 1.153518557548523 | 1.153518557548523 | 1.153518557548523 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4650519788265228 | 0.4650519788265228 | 0.4650519788265228 |
| pearson_corr | 0.6341341137886047 | 0.6341341137886047 | 0.6341341137886047 |
| jensen_shannon_divergence | 1.1759909391403198 | 1.1759909391403198 | 1.1759909391403198 |


----------------------------



### 0.001, tanh, softmax

In [4]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='tanh')(in_layer)
        net = layers.Dense(64, activation='tanh')(net)
        net = layers.Dense(32, activation='tanh')(net)
        net = layers.Dense(16, activation='tanh')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='softmax')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7031755447387695 | 0.7031755447387695 | 0.7031755447387695 |
| pearson_corr | 0.5129069089889526 | 0.5129069089889526 | 0.5129069089889526 |
| jensen_shannon_divergence | 10.174050331115723 | 10.174050331115723 | 10.174050331115723 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5595499277114868 | 0.5595499277114868 | 0.5595499277114868 |
| pearson_corr | 0.41260793805122375 | 0.41260793805122375 | 0.41260793805122375 |
| jensen_shannon_divergence | 1.81981360912323 | 1.81981360912323 | 1.81981360912323 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.805553674697876 | 0.805553674697876 | 0.805553674697876 |
| pearson_corr | 0.05302850157022476 | 0.05302850157022476 | 0.05302850157022476 |
| jensen_shannon_divergence | 3.495584726333618 | 3.495584726333618 | 3.495584726333618 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.452847957611084 | 0.452847957611084 | 0.452847957611084 |
| pearson_corr | 0.6587474346160889 | 0.6587474346160889 | 0.6587474346160889 |
| jensen_shannon_divergence | 1.1684390306472778 | 1.1684390306472778 | 1.1684390306472778 |


----------------------------



### 0.001, tanh, sigmoid

In [5]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='tanh')(in_layer)
        net = layers.Dense(64, activation='tanh')(net)
        net = layers.Dense(32, activation='tanh')(net)
        net = layers.Dense(16, activation='tanh')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='sigmoid')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7322008013725281 | 0.7322008013725281 | 0.7322008013725281 |
| pearson_corr | 0.5039581060409546 | 0.5039581060409546 | 0.5039581060409546 |
| jensen_shannon_divergence | 10.517875671386719 | 10.517875671386719 | 10.517875671386719 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5354747176170349 | 0.5354747176170349 | 0.5354747176170349 |
| pearson_corr | 0.5559419393539429 | 0.5559419393539429 | 0.5559419393539429 |
| jensen_shannon_divergence | 1.471104383468628 | 1.471104383468628 | 1.471104383468628 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8085001707077026 | 0.8085001707077026 | 0.8085001707077026 |
| pearson_corr | 0.06993252784013748 | 0.06993252784013748 | 0.06993252784013748 |
| jensen_shannon_divergence | 4.0049638748168945 | 4.0049638748168945 | 4.0049638748168945 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4716844856739044 | 0.4716844856739044 | 0.4716844856739044 |
| pearson_corr | 0.6332216262817383 | 0.6332216262817383 | 0.6332216262817383 |
| jensen_shannon_divergence | 1.1996990442276 | 1.1996990442276 | 1.1996990442276 |


----------------------------



### 0.001, tanh, relu

In [6]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='tanh')(in_layer)
        net = layers.Dense(64, activation='tanh')(net)
        net = layers.Dense(32, activation='tanh')(net)
        net = layers.Dense(16, activation='tanh')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='relu')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.725777804851532 | 0.725777804851532 | 0.725777804851532 |
| pearson_corr | 0.5115144848823547 | 0.5115144848823547 | 0.5115144848823547 |
| jensen_shannon_divergence | 10.507181167602539 | 10.507181167602539 | 10.507181167602539 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5212878584861755 | 0.5212878584861755 | 0.5212878584861755 |
| pearson_corr | 0.5815295577049255 | 0.5815295577049255 | 0.5815295577049255 |
| jensen_shannon_divergence | 1.361104965209961 | 1.361104965209961 | 1.361104965209961 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8060070872306824 | 0.8060070872306824 | 0.8060070872306824 |
| pearson_corr | 0.07296964526176453 | 0.07296964526176453 | 0.07296964526176453 |
| jensen_shannon_divergence | 3.9711387157440186 | 3.9711387157440186 | 3.9711387157440186 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.581969141960144 | 0.581969141960144 | 0.581969141960144 |
| pearson_corr | 0.4674793779850006 | 0.4674793779850006 | 0.4674793779850006 |
| jensen_shannon_divergence | 1.7031594514846802 | 1.7031594514846802 | 1.7031594514846802 |


----------------------------



### 0.001, softmax, tanh

In [7]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
  
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='softmax')(in_layer)
        net = layers.Dense(64, activation='softmax')(net)
        net = layers.Dense(32, activation='softmax')(net)
        net = layers.Dense(16, activation='softmax')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='tanh')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.689258873462677 | 0.689258873462677 | 0.689258873462677 |
| pearson_corr | 0.5604363083839417 | 0.5604363083839417 | 0.5604363083839417 |
| jensen_shannon_divergence | 10.813039779663086 | 10.813039779663086 | 10.813039779663086 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4751291573047638 | 0.4751291573047638 | 0.4751291573047638 |
| pearson_corr | 0.6314650774002075 | 0.6314650774002075 | 0.6314650774002075 |
| jensen_shannon_divergence | 1.3048005104064941 | 1.3048005104064941 | 1.3048005104064941 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4735287129878998 | 0.4735287129878998 | 0.4735287129878998 |
| pearson_corr | 0.6518118381500244 | 0.6518118381500244 | 0.6518118381500244 |
| jensen_shannon_divergence | 1.206734538078308 | 1.206734538078308 | 1.206734538078308 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4725693166255951 | 0.4725693166255951 | 0.4725693166255951 |
| pearson_corr | 0.6286318898200989 | 0.6286318898200989 | 0.6286318898200989 |
| jensen_shannon_divergence | 1.2111378908157349 | 1.2111378908157349 | 1.2111378908157349 |


----------------------------



### 0.001, softmax, softmax

In [8]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')

    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='softmax')(in_layer)
        net = layers.Dense(64, activation='softmax')(net)
        net = layers.Dense(32, activation='softmax')(net)
        net = layers.Dense(16, activation='softmax')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='softmax')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7021464705467224 | 0.7021464705467224 | 0.7021464705467224 |
| pearson_corr | 0.5168219804763794 | 0.5168219804763794 | 0.5168219804763794 |
| jensen_shannon_divergence | 10.154905319213867 | 10.154905319213867 | 10.154905319213867 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.568131148815155 | 0.568131148815155 | 0.568131148815155 |
| pearson_corr | 0.38372287154197693 | 0.38372287154197693 | 0.38372287154197693 |
| jensen_shannon_divergence | 1.8875412940979004 | 1.8875412940979004 | 1.8875412940979004 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8064106106758118 | 0.8064106106758118 | 0.8064106106758118 |
| pearson_corr | 0.049776118248701096 | 0.049776118248701096 | 0.049776118248701096 |
| jensen_shannon_divergence | 3.5086987018585205 | 3.5086987018585205 | 3.5086987018585205 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4729299247264862 | 0.4729299247264862 | 0.4729299247264862 |
| pearson_corr | 0.627537190914154 | 0.627537190914154 | 0.627537190914154 |
| jensen_shannon_divergence | 1.2074182033538818 | 1.2074182033538818 | 1.2074182033538818 |


----------------------------



### 0.001, softmax, sigmoid

In [9]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')

    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='softmax')(in_layer)
        net = layers.Dense(64, activation='softmax')(net)
        net = layers.Dense(32, activation='softmax')(net)
        net = layers.Dense(16, activation='softmax')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='sigmoid')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7372624278068542 | 0.7372624278068542 | 0.7372624278068542 |
| pearson_corr | 0.49875739216804504 | 0.49875739216804504 | 0.49875739216804504 |
| jensen_shannon_divergence | 10.495192527770996 | 10.495192527770996 | 10.495192527770996 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5259911417961121 | 0.5259911417961121 | 0.5259911417961121 |
| pearson_corr | 0.5579653978347778 | 0.5579653978347778 | 0.5579653978347778 |
| jensen_shannon_divergence | 1.4855825901031494 | 1.4855825901031494 | 1.4855825901031494 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.815433919429779 | 0.815433919429779 | 0.815433919429779 |
| pearson_corr | 0.06017526239156723 | 0.06017526239156723 | 0.06017526239156723 |
| jensen_shannon_divergence | 4.164823055267334 | 4.164823055267334 | 4.164823055267334 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.47948509454727173 | 0.47948509454727173 | 0.47948509454727173 |
| pearson_corr | 0.6286799907684326 | 0.6286799907684326 | 0.6286799907684326 |
| jensen_shannon_divergence | 1.2274622917175293 | 1.2274622917175293 | 1.2274622917175293 |


----------------------------



### 0.001, softmax, relu

In [10]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')


    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='softmax')(in_layer)
        net = layers.Dense(64, activation='softmax')(net)
        net = layers.Dense(32, activation='softmax')(net)
        net = layers.Dense(16, activation='softmax')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='relu')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.777326226234436 | 0.777326226234436 | 0.777326226234436 |
| pearson_corr | 0.47031328082084656 | 0.47031328082084656 | 0.47031328082084656 |
| jensen_shannon_divergence | 10.046243667602539 | 10.046243667602539 | 10.046243667602539 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5227521657943726 | 0.5227521657943726 | 0.5227521657943726 |
| pearson_corr | 0.5831286907196045 | 0.5831286907196045 | 0.5831286907196045 |
| jensen_shannon_divergence | 1.38713800907135 | 1.38713800907135 | 1.38713800907135 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8545444011688232 | 0.8545444011688232 | 0.8545444011688232 |
| pearson_corr | 0.01872345805168152 | 0.01872345805168152 | 0.01872345805168152 |
| jensen_shannon_divergence | 4.803750038146973 | 4.803750038146973 | 4.803750038146973 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6102150082588196 | 0.6102150082588196 | 0.6102150082588196 |
| pearson_corr | 0.41744521260261536 | 0.41744521260261536 | 0.41744521260261536 |
| jensen_shannon_divergence | 1.6879318952560425 | 1.6879318952560425 | 1.6879318952560425 |


----------------------------



### 0.001, sigmoid, tanh

In [11]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')

    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='sigmoid')(in_layer)
        net = layers.Dense(64, activation='sigmoid')(net)
        net = layers.Dense(32, activation='sigmoid')(net)
        net = layers.Dense(16, activation='sigmoid')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='tanh')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6895681619644165 | 0.6895681619644165 | 0.6895681619644165 |
| pearson_corr | 0.5592795014381409 | 0.5592795014381409 | 0.5592795014381409 |
| jensen_shannon_divergence | 10.812594413757324 | 10.812594413757324 | 10.812594413757324 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4727245569229126 | 0.4727245569229126 | 0.4727245569229126 |
| pearson_corr | 0.6368755102157593 | 0.6368755102157593 | 0.6368755102157593 |
| jensen_shannon_divergence | 1.2965010404586792 | 1.2965010404586792 | 1.2965010404586792 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4764680564403534 | 0.4764680564403534 | 0.4764680564403534 |
| pearson_corr | 0.649135947227478 | 0.649135947227478 | 0.649135947227478 |
| jensen_shannon_divergence | 1.2082154750823975 | 1.2082154750823975 | 1.2082154750823975 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4606320858001709 | 0.4606320858001709 | 0.4606320858001709 |
| pearson_corr | 0.6512191891670227 | 0.6512191891670227 | 0.6512191891670227 |
| jensen_shannon_divergence | 1.1895689964294434 | 1.1895689964294434 | 1.1895689964294434 |


----------------------------



### 0.001, sigmoid, softmax

In [12]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')

    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='sigmoid')(in_layer)
        net = layers.Dense(64, activation='sigmoid')(net)
        net = layers.Dense(32, activation='sigmoid')(net)
        net = layers.Dense(16, activation='sigmoid')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='softmax')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7050991654396057 | 0.7050991654396057 | 0.7050991654396057 |
| pearson_corr | 0.5054289102554321 | 0.5054289102554321 | 0.5054289102554321 |
| jensen_shannon_divergence | 10.200511932373047 | 10.200511932373047 | 10.200511932373047 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.570213794708252 | 0.570213794708252 | 0.570213794708252 |
| pearson_corr | 0.3781324326992035 | 0.3781324326992035 | 0.3781324326992035 |
| jensen_shannon_divergence | 1.902363896369934 | 1.902363896369934 | 1.902363896369934 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8058650493621826 | 0.8058650493621826 | 0.8058650493621826 |
| pearson_corr | 0.05199955403804779 | 0.05199955403804779 | 0.05199955403804779 |
| jensen_shannon_divergence | 3.4998390674591064 | 3.4998390674591064 | 3.4998390674591064 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4518149793148041 | 0.4518149793148041 | 0.4518149793148041 |
| pearson_corr | 0.6599947214126587 | 0.6599947214126587 | 0.6599947214126587 |
| jensen_shannon_divergence | 1.1504522562026978 | 1.1504522562026978 | 1.1504522562026978 |


----------------------------



### 0.001, sigmoid, sigmoid

In [13]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')

    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='sigmoid')(in_layer)
        net = layers.Dense(64, activation='sigmoid')(net)
        net = layers.Dense(32, activation='sigmoid')(net)
        net = layers.Dense(16, activation='sigmoid')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='sigmoid')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7322612404823303 | 0.7322612404823303 | 0.7322612404823303 |
| pearson_corr | 0.5038964152336121 | 0.5038964152336121 | 0.5038964152336121 |
| jensen_shannon_divergence | 10.517156600952148 | 10.517156600952148 | 10.517156600952148 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5343048572540283 | 0.5343048572540283 | 0.5343048572540283 |
| pearson_corr | 0.5597256422042847 | 0.5597256422042847 | 0.5597256422042847 |
| jensen_shannon_divergence | 1.4634602069854736 | 1.4634602069854736 | 1.4634602069854736 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8079275488853455 | 0.8079275488853455 | 0.8079275488853455 |
| pearson_corr | 0.07143078744411469 | 0.07143078744411469 | 0.07143078744411469 |
| jensen_shannon_divergence | 3.9873385429382324 | 3.9873385429382324 | 3.9873385429382324 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4729439616203308 | 0.4729439616203308 | 0.4729439616203308 |
| pearson_corr | 0.6275655627250671 | 0.6275655627250671 | 0.6275655627250671 |
| jensen_shannon_divergence | 1.2078404426574707 | 1.2078404426574707 | 1.2078404426574707 |


----------------------------



### 0.001, sigmoid, relu

In [14]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='sigmoid')(in_layer)
        net = layers.Dense(64, activation='sigmoid')(net)
        net = layers.Dense(32, activation='sigmoid')(net)
        net = layers.Dense(16, activation='sigmoid')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='relu')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6956695914268494 | 0.6956695914268494 | 0.6956695914268494 |
| pearson_corr | 0.5383015275001526 | 0.5383015275001526 | 0.5383015275001526 |
| jensen_shannon_divergence | 10.29437255859375 | 10.29437255859375 | 10.29437255859375 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5298888683319092 | 0.5298888683319092 | 0.5298888683319092 |
| pearson_corr | 0.5638238787651062 | 0.5638238787651062 | 0.5638238787651062 |
| jensen_shannon_divergence | 1.4051779508590698 | 1.4051779508590698 | 1.4051779508590698 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8635128736495972 | 0.8635128736495972 | 0.8635128736495972 |
| pearson_corr | 0.0028945652302354574 | 0.0028945652302354574 | 0.0028945652302354574 |
| jensen_shannon_divergence | 4.916763782501221 | 4.916763782501221 | 4.916763782501221 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6301590800285339 | 0.6301590800285339 | 0.6301590800285339 |
| pearson_corr | 0.37271249294281006 | 0.37271249294281006 | 0.37271249294281006 |
| jensen_shannon_divergence | 1.8220829963684082 | 1.8220829963684082 | 1.8220829963684082 |


----------------------------



### 0.001, relu, tanh

In [15]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='relu')(in_layer)
        net = layers.Dense(64, activation='relu')(net)
        net = layers.Dense(32, activation='relu')(net)
        net = layers.Dense(16, activation='relu')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='tanh')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7178373336791992 | 0.7178373336791992 | 0.7178373336791992 |
| pearson_corr | 0.520713210105896 | 0.520713210105896 | 0.520713210105896 |
| jensen_shannon_divergence | 11.09044361114502 | 11.09044361114502 | 11.09044361114502 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.9995865225791931 | 0.9995865225791931 | 0.9995865225791931 |
| pearson_corr | -0.011955349706113338 | -0.011955349706113338 | -0.011955349706113338 |
| jensen_shannon_divergence | 14.03266429901123 | 14.03266429901123 | 14.03266429901123 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7665342092514038 | 0.7665342092514038 | 0.7665342092514038 |
| pearson_corr | 0.3188605010509491 | 0.3188605010509491 | 0.3188605010509491 |
| jensen_shannon_divergence | 4.051032543182373 | 4.051032543182373 | 4.051032543182373 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.9622381329536438 | 0.9622381329536438 | 0.9622381329536438 |
| pearson_corr | 0.05282896012067795 | 0.05282896012067795 | 0.05282896012067795 |
| jensen_shannon_divergence | 11.985322952270508 | 11.985322952270508 | 11.985322952270508 |


----------------------------



### 0.001, relu, softmax

In [16]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='relu')(in_layer)
        net = layers.Dense(64, activation='relu')(net)
        net = layers.Dense(32, activation='relu')(net)
        net = layers.Dense(16, activation='relu')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='softmax')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7649039626121521 | 0.7649039626121521 | 0.7649039626121521 |
| pearson_corr | 0.4780537486076355 | 0.4780537486076355 | 0.4780537486076355 |
| jensen_shannon_divergence | 8.77587890625 | 8.77587890625 | 8.77587890625 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7835187911987305 | 0.7835187911987305 | 0.7835187911987305 |
| pearson_corr | 0.1345556676387787 | 0.1345556676387787 | 0.1345556676387787 |
| jensen_shannon_divergence | 3.5534567832946777 | 3.5534567832946777 | 3.5534567832946777 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.9141563177108765 | 0.9141563177108765 | 0.9141563177108765 |
| pearson_corr | -0.0014357281615957618 | -0.0014357281615957618 | -0.0014357281615957618 |
| jensen_shannon_divergence | 6.227056503295898 | 6.227056503295898 | 6.227056503295898 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.9909206032752991 | 0.9909206032752991 | 0.9909206032752991 |
| pearson_corr | 0.00823667086660862 | 0.00823667086660862 | 0.00823667086660862 |
| jensen_shannon_divergence | 10.616567611694336 | 10.616567611694336 | 10.616567611694336 |


----------------------------



### 0.001, relu, sigmoid

In [17]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='relu')(in_layer)
        net = layers.Dense(64, activation='relu')(net)
        net = layers.Dense(32, activation='relu')(net)
        net = layers.Dense(16, activation='relu')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='sigmoid')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8032684922218323 | 0.8032684922218323 | 0.8032684922218323 |
| pearson_corr | 0.3813827931880951 | 0.3813827931880951 | 0.3813827931880951 |
| jensen_shannon_divergence | 5.960070610046387 | 5.960070610046387 | 5.960070610046387 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8256418704986572 | 0.8256418704986572 | 0.8256418704986572 |
| pearson_corr | 0.11947380006313324 | 0.11947380006313324 | 0.11947380006313324 |
| jensen_shannon_divergence | 5.856802940368652 | 5.856802940368652 | 5.856802940368652 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.9104326963424683 | 0.9104326963424683 | 0.9104326963424683 |
| pearson_corr | -0.047585904598236084 | -0.047585904598236084 | -0.047585904598236084 |
| jensen_shannon_divergence | 5.851060390472412 | 5.851060390472412 | 5.851060390472412 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.9821526408195496 | 0.9821526408195496 | 0.9821526408195496 |
| pearson_corr | -0.005772305186837912 | -0.005772305186837912 | -0.005772305186837912 |
| jensen_shannon_divergence | 11.031747817993164 | 11.031747817993164 | 11.031747817993164 |


----------------------------



### 0.001, relu, relu

In [18]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='relu')(in_layer)
        net = layers.Dense(64, activation='relu')(net)
        net = layers.Dense(32, activation='relu')(net)
        net = layers.Dense(16, activation='relu')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='relu')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7380648851394653 | 0.7380648851394653 | 0.7380648851394653 |
| pearson_corr | 0.43702155351638794 | 0.43702155351638794 | 0.43702155351638794 |
| jensen_shannon_divergence | 9.53736686706543 | 9.53736686706543 | 9.53736686706543 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5425742268562317 | 0.5425742268562317 | 0.5425742268562317 |
| pearson_corr | 0.5464356541633606 | 0.5464356541633606 | 0.5464356541633606 |
| jensen_shannon_divergence | 1.5832644701004028 | 1.5832644701004028 | 1.5832644701004028 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8444424867630005 | 0.8444424867630005 | 0.8444424867630005 |
| pearson_corr | -0.20108038187026978 | -0.20108038187026978 | -0.20108038187026978 |
| jensen_shannon_divergence | 4.037468910217285 | 4.037468910217285 | 4.037468910217285 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5227006077766418 | 0.5227006077766418 | 0.5227006077766418 |
| pearson_corr | 0.6006878018379211 | 0.6006878018379211 | 0.6006878018379211 |
| jensen_shannon_divergence | 1.959267020225525 | 1.959267020225525 | 1.959267020225525 |


----------------------------



---------------------------
### 0.01, tanh, tanh

In [19]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='tanh')(in_layer)
        net = layers.Dense(64, activation='tanh')(net)
        net = layers.Dense(32, activation='tanh')(net)
        net = layers.Dense(16, activation='tanh')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='tanh')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6891995668411255 | 0.6891995668411255 | 0.6891995668411255 |
| pearson_corr | 0.5610076785087585 | 0.5610076785087585 | 0.5610076785087585 |
| jensen_shannon_divergence | 10.815064430236816 | 10.815064430236816 | 10.815064430236816 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4804842472076416 | 0.4804842472076416 | 0.4804842472076416 |
| pearson_corr | 0.6177962422370911 | 0.6177962422370911 | 0.6177962422370911 |
| jensen_shannon_divergence | 1.4304665327072144 | 1.4304665327072144 | 1.4304665327072144 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4603343605995178 | 0.4603343605995178 | 0.4603343605995178 |
| pearson_corr | 0.6541933417320251 | 0.6541933417320251 | 0.6541933417320251 |
| jensen_shannon_divergence | 1.2058701515197754 | 1.2058701515197754 | 1.2058701515197754 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5508818030357361 | 0.5508818030357361 | 0.5508818030357361 |
| pearson_corr | 0.5445223450660706 | 0.5445223450660706 | 0.5445223450660706 |
| jensen_shannon_divergence | 1.6750348806381226 | 1.6750348806381226 | 1.6750348806381226 |


----------------------------



### 0.01, tanh, softmax

In [20]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='tanh')(in_layer)
        net = layers.Dense(64, activation='tanh')(net)
        net = layers.Dense(32, activation='tanh')(net)
        net = layers.Dense(16, activation='tanh')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='softmax')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7025334239006042 | 0.7025334239006042 | 0.7025334239006042 |
| pearson_corr | 0.5153740048408508 | 0.5153740048408508 | 0.5153740048408508 |
| jensen_shannon_divergence | 10.165254592895508 | 10.165254592895508 | 10.165254592895508 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5712583661079407 | 0.5712583661079407 | 0.5712583661079407 |
| pearson_corr | 0.3777683675289154 | 0.3777683675289154 | 0.3777683675289154 |
| jensen_shannon_divergence | 1.8967208862304688 | 1.8967208862304688 | 1.8967208862304688 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8063222765922546 | 0.8063222765922546 | 0.8063222765922546 |
| pearson_corr | 0.050953447818756104 | 0.050953447818756104 | 0.050953447818756104 |
| jensen_shannon_divergence | 3.504469871520996 | 3.504469871520996 | 3.504469871520996 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.46426257491111755 | 0.46426257491111755 | 0.46426257491111755 |
| pearson_corr | 0.6482221484184265 | 0.6482221484184265 | 0.6482221484184265 |
| jensen_shannon_divergence | 1.1906579732894897 | 1.1906579732894897 | 1.1906579732894897 |


----------------------------



### 0.01, tanh, sigmoid

In [21]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='tanh')(in_layer)
        net = layers.Dense(64, activation='tanh')(net)
        net = layers.Dense(32, activation='tanh')(net)
        net = layers.Dense(16, activation='tanh')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='sigmoid')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7343807816505432 | 0.7343807816505432 | 0.7343807816505432 |
| pearson_corr | 0.5017017722129822 | 0.5017017722129822 | 0.5017017722129822 |
| jensen_shannon_divergence | 10.521632194519043 | 10.521632194519043 | 10.521632194519043 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.537043035030365 | 0.537043035030365 | 0.537043035030365 |
| pearson_corr | 0.5553240180015564 | 0.5553240180015564 | 0.5553240180015564 |
| jensen_shannon_divergence | 1.4690927267074585 | 1.4690927267074585 | 1.4690927267074585 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8066955804824829 | 0.8066955804824829 | 0.8066955804824829 |
| pearson_corr | 0.07180380821228027 | 0.07180380821228027 | 0.07180380821228027 |
| jensen_shannon_divergence | 3.9661641120910645 | 3.9661641120910645 | 3.9661641120910645 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4599783420562744 | 0.4599783420562744 | 0.4599783420562744 |
| pearson_corr | 0.649744987487793 | 0.649744987487793 | 0.649744987487793 |
| jensen_shannon_divergence | 1.1734665632247925 | 1.1734665632247925 | 1.1734665632247925 |


----------------------------



### 0.01, tanh, relu

In [22]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='tanh')(in_layer)
        net = layers.Dense(64, activation='tanh')(net)
        net = layers.Dense(32, activation='tanh')(net)
        net = layers.Dense(16, activation='tanh')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='relu')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7360832691192627 | 0.7360832691192627 | 0.7360832691192627 |
| pearson_corr | 0.4998616576194763 | 0.4998616576194763 | 0.4998616576194763 |
| jensen_shannon_divergence | 10.524311065673828 | 10.524311065673828 | 10.524311065673828 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5268705487251282 | 0.5268705487251282 | 0.5268705487251282 |
| pearson_corr | 0.5651369094848633 | 0.5651369094848633 | 0.5651369094848633 |
| jensen_shannon_divergence | 1.377767562866211 | 1.377767562866211 | 1.377767562866211 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8088623285293579 | 0.8088623285293579 | 0.8088623285293579 |
| pearson_corr | 0.06751136481761932 | 0.06751136481761932 | 0.06751136481761932 |
| jensen_shannon_divergence | 4.037070274353027 | 4.037070274353027 | 4.037070274353027 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4933841824531555 | 0.4933841824531555 | 0.4933841824531555 |
| pearson_corr | 0.6314826607704163 | 0.6314826607704163 | 0.6314826607704163 |
| jensen_shannon_divergence | 1.2306410074234009 | 1.2306410074234009 | 1.2306410074234009 |


----------------------------



### 0.01, softmax, tanh

In [23]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='softmax')(in_layer)
        net = layers.Dense(64, activation='softmax')(net)
        net = layers.Dense(32, activation='softmax')(net)
        net = layers.Dense(16, activation='softmax')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='tanh')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.689617395401001 | 0.689617395401001 | 0.689617395401001 |
| pearson_corr | 0.5590711832046509 | 0.5590711832046509 | 0.5590711832046509 |
| jensen_shannon_divergence | 10.812705993652344 | 10.812705993652344 | 10.812705993652344 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.48050469160079956 | 0.48050469160079956 | 0.48050469160079956 |
| pearson_corr | 0.6258462071418762 | 0.6258462071418762 | 0.6258462071418762 |
| jensen_shannon_divergence | 1.323761224746704 | 1.323761224746704 | 1.323761224746704 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4771098792552948 | 0.4771098792552948 | 0.4771098792552948 |
| pearson_corr | 0.6485029458999634 | 0.6485029458999634 | 0.6485029458999634 |
| jensen_shannon_divergence | 1.2282942533493042 | 1.2282942533493042 | 1.2282942533493042 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.47863611578941345 | 0.47863611578941345 | 0.47863611578941345 |
| pearson_corr | 0.6157286167144775 | 0.6157286167144775 | 0.6157286167144775 |
| jensen_shannon_divergence | 1.2715953588485718 | 1.2715953588485718 | 1.2715953588485718 |


----------------------------



### 0.01, softmax, softmax

In [24]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='softmax')(in_layer)
        net = layers.Dense(64, activation='softmax')(net)
        net = layers.Dense(32, activation='softmax')(net)
        net = layers.Dense(16, activation='softmax')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='softmax')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7030878067016602 | 0.7030878067016602 | 0.7030878067016602 |
| pearson_corr | 0.5132420063018799 | 0.5132420063018799 | 0.5132420063018799 |
| jensen_shannon_divergence | 10.172938346862793 | 10.172938346862793 | 10.172938346862793 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5695521831512451 | 0.5695521831512451 | 0.5695521831512451 |
| pearson_corr | 0.38117513060569763 | 0.38117513060569763 | 0.38117513060569763 |
| jensen_shannon_divergence | 1.8952945470809937 | 1.8952945470809937 | 1.8952945470809937 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.80597323179245 | 0.80597323179245 | 0.80597323179245 |
| pearson_corr | 0.05217134207487106 | 0.05217134207487106 | 0.05217134207487106 |
| jensen_shannon_divergence | 3.499063730239868 | 3.499063730239868 | 3.499063730239868 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4720597267150879 | 0.4720597267150879 | 0.4720597267150879 |
| pearson_corr | 0.6292254328727722 | 0.6292254328727722 | 0.6292254328727722 |
| jensen_shannon_divergence | 1.2078742980957031 | 1.2078742980957031 | 1.2078742980957031 |


----------------------------



### 0.01, softmax, sigmoid

In [25]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='softmax')(in_layer)
        net = layers.Dense(64, activation='softmax')(net)
        net = layers.Dense(32, activation='softmax')(net)
        net = layers.Dense(16, activation='softmax')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='sigmoid')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7320654988288879 | 0.7320654988288879 | 0.7320654988288879 |
| pearson_corr | 0.5040974020957947 | 0.5040974020957947 | 0.5040974020957947 |
| jensen_shannon_divergence | 10.518004417419434 | 10.518004417419434 | 10.518004417419434 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5361037254333496 | 0.5361037254333496 | 0.5361037254333496 |
| pearson_corr | 0.5557147860527039 | 0.5557147860527039 | 0.5557147860527039 |
| jensen_shannon_divergence | 1.4697747230529785 | 1.4697747230529785 | 1.4697747230529785 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.808158278465271 | 0.808158278465271 | 0.808158278465271 |
| pearson_corr | 0.07047773897647858 | 0.07047773897647858 | 0.07047773897647858 |
| jensen_shannon_divergence | 3.9960293769836426 | 3.9960293769836426 | 3.9960293769836426 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.47314926981925964 | 0.47314926981925964 | 0.47314926981925964 |
| pearson_corr | 0.6277148723602295 | 0.6277148723602295 | 0.6277148723602295 |
| jensen_shannon_divergence | 1.2065932750701904 | 1.2065932750701904 | 1.2065932750701904 |


----------------------------



### 0.01, softmax, relu

In [26]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='softmax')(in_layer)
        net = layers.Dense(64, activation='softmax')(net)
        net = layers.Dense(32, activation='softmax')(net)
        net = layers.Dense(16, activation='softmax')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='relu')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7784123420715332 | 0.7784123420715332 | 0.7784123420715332 |
| pearson_corr | 0.46971264481544495 | 0.46971264481544495 | 0.46971264481544495 |
| jensen_shannon_divergence | 10.036648750305176 | 10.036648750305176 | 10.036648750305176 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5268705487251282 | 0.5268705487251282 | 0.5268705487251282 |
| pearson_corr | 0.5651369094848633 | 0.5651369094848633 | 0.5651369094848633 |
| jensen_shannon_divergence | 1.377767562866211 | 1.377767562866211 | 1.377767562866211 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8535595536231995 | 0.8535595536231995 | 0.8535595536231995 |
| pearson_corr | 0.019253190606832504 | 0.019253190606832504 | 0.019253190606832504 |
| jensen_shannon_divergence | 4.787984848022461 | 4.787984848022461 | 4.787984848022461 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5980827212333679 | 0.5980827212333679 | 0.5980827212333679 |
| pearson_corr | 0.43147265911102295 | 0.43147265911102295 | 0.43147265911102295 |
| jensen_shannon_divergence | 1.6222798824310303 | 1.6222798824310303 | 1.6222798824310303 |


----------------------------



### 0.01, sigmoid, tanh

In [27]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='sigmoid')(in_layer)
        net = layers.Dense(64, activation='sigmoid')(net)
        net = layers.Dense(32, activation='sigmoid')(net)
        net = layers.Dense(16, activation='sigmoid')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='tanh')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6890764236450195 | 0.6890764236450195 | 0.6890764236450195 |
| pearson_corr | 0.5609753131866455 | 0.5609753131866455 | 0.5609753131866455 |
| jensen_shannon_divergence | 10.813846588134766 | 10.813846588134766 | 10.813846588134766 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.48470062017440796 | 0.48470062017440796 | 0.48470062017440796 |
| pearson_corr | 0.621523380279541 | 0.621523380279541 | 0.621523380279541 |
| jensen_shannon_divergence | 1.3875843286514282 | 1.3875843286514282 | 1.3875843286514282 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4861123859882355 | 0.4861123859882355 | 0.4861123859882355 |
| pearson_corr | 0.6384822726249695 | 0.6384822726249695 | 0.6384822726249695 |
| jensen_shannon_divergence | 1.245298981666565 | 1.245298981666565 | 1.245298981666565 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.47816288471221924 | 0.47816288471221924 | 0.47816288471221924 |
| pearson_corr | 0.6309295296669006 | 0.6309295296669006 | 0.6309295296669006 |
| jensen_shannon_divergence | 1.2289056777954102 | 1.2289056777954102 | 1.2289056777954102 |


----------------------------



### 0.01, sigmoid, softmax

In [28]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='sigmoid')(in_layer)
        net = layers.Dense(64, activation='sigmoid')(net)
        net = layers.Dense(32, activation='sigmoid')(net)
        net = layers.Dense(16, activation='sigmoid')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='softmax')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7029302716255188 | 0.7029302716255188 | 0.7029302716255188 |
| pearson_corr | 0.5138469338417053 | 0.5138469338417053 | 0.5138469338417053 |
| jensen_shannon_divergence | 10.171008110046387 | 10.171008110046387 | 10.171008110046387 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5673391222953796 | 0.5673391222953796 | 0.5673391222953796 |
| pearson_corr | 0.3931182324886322 | 0.3931182324886322 | 0.3931182324886322 |
| jensen_shannon_divergence | 1.869105577468872 | 1.869105577468872 | 1.869105577468872 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8087310194969177 | 0.8087310194969177 | 0.8087310194969177 |
| pearson_corr | 0.04573727026581764 | 0.04573727026581764 | 0.04573727026581764 |
| jensen_shannon_divergence | 3.525860071182251 | 3.525860071182251 | 3.525860071182251 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.46102017164230347 | 0.46102017164230347 | 0.46102017164230347 |
| pearson_corr | 0.6443154215812683 | 0.6443154215812683 | 0.6443154215812683 |
| jensen_shannon_divergence | 1.1718589067459106 | 1.1718589067459106 | 1.1718589067459106 |


----------------------------



### 0.01, sigmoid, sigmoid

In [29]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='sigmoid')(in_layer)
        net = layers.Dense(64, activation='sigmoid')(net)
        net = layers.Dense(32, activation='sigmoid')(net)
        net = layers.Dense(16, activation='sigmoid')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='sigmoid')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7331469058990479 | 0.7331469058990479 | 0.7331469058990479 |
| pearson_corr | 0.5029808282852173 | 0.5029808282852173 | 0.5029808282852173 |
| jensen_shannon_divergence | 10.519803047180176 | 10.519803047180176 | 10.519803047180176 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5358425378799438 | 0.5358425378799438 | 0.5358425378799438 |
| pearson_corr | 0.5600013732910156 | 0.5600013732910156 | 0.5600013732910156 |
| jensen_shannon_divergence | 1.4596737623214722 | 1.4596737623214722 | 1.4596737623214722 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8073012232780457 | 0.8073012232780457 | 0.8073012232780457 |
| pearson_corr | 0.07074924558401108 | 0.07074924558401108 | 0.07074924558401108 |
| jensen_shannon_divergence | 3.9809534549713135 | 3.9809534549713135 | 3.9809534549713135 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4739166796207428 | 0.4739166796207428 | 0.4739166796207428 |
| pearson_corr | 0.6260849833488464 | 0.6260849833488464 | 0.6260849833488464 |
| jensen_shannon_divergence | 1.2186535596847534 | 1.2186535596847534 | 1.2186535596847534 |


----------------------------



### 0.01, sigmoid, relu

In [30]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='sigmoid')(in_layer)
        net = layers.Dense(64, activation='sigmoid')(net)
        net = layers.Dense(32, activation='sigmoid')(net)
        net = layers.Dense(16, activation='sigmoid')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='relu')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.717445433139801 | 0.717445433139801 | 0.717445433139801 |
| pearson_corr | 0.47136104106903076 | 0.47136104106903076 | 0.47136104106903076 |
| jensen_shannon_divergence | 10.465052604675293 | 10.465052604675293 | 10.465052604675293 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5268705487251282 | 0.5268705487251282 | 0.5268705487251282 |
| pearson_corr | 0.5651369094848633 | 0.5651369094848633 | 0.5651369094848633 |
| jensen_shannon_divergence | 1.377767562866211 | 1.377767562866211 | 1.377767562866211 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8622609376907349 | 0.8622609376907349 | 0.8622609376907349 |
| pearson_corr | 0.002742087934166193 | 0.002742087934166193 | 0.002742087934166193 |
| jensen_shannon_divergence | 4.876488208770752 | 4.876488208770752 | 4.876488208770752 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5225847363471985 | 0.5225847363471985 | 0.5225847363471985 |
| pearson_corr | 0.5588651895523071 | 0.5588651895523071 | 0.5588651895523071 |
| jensen_shannon_divergence | 1.4375776052474976 | 1.4375776052474976 | 1.4375776052474976 |


----------------------------



### 0.01, relu, tanh

In [31]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='relu')(in_layer)
        net = layers.Dense(64, activation='relu')(net)
        net = layers.Dense(32, activation='relu')(net)
        net = layers.Dense(16, activation='relu')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='tanh')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7178373336791992 | 0.7178373336791992 | 0.7178373336791992 |
| pearson_corr | 0.520713210105896 | 0.520713210105896 | 0.520713210105896 |
| jensen_shannon_divergence | 11.09044361114502 | 11.09044361114502 | 11.09044361114502 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.9890556931495667 | 0.9890556931495667 | 0.9890556931495667 |
| pearson_corr | -0.01377143245190382 | -0.01377143245190382 | -0.01377143245190382 |
| jensen_shannon_divergence | 11.924043655395508 | 11.924043655395508 | 11.924043655395508 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7665342092514038 | 0.7665342092514038 | 0.7665342092514038 |
| pearson_corr | 0.3188605010509491 | 0.3188605010509491 | 0.3188605010509491 |
| jensen_shannon_divergence | 4.051032543182373 | 4.051032543182373 | 4.051032543182373 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.9728819131851196 | 0.9728819131851196 | 0.9728819131851196 |
| pearson_corr | 0.013229726813733578 | 0.013229726813733578 | 0.013229726813733578 |
| jensen_shannon_divergence | 12.99627685546875 | 12.99627685546875 | 12.99627685546875 |


----------------------------



### 0.01, relu, softmax

In [32]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='relu')(in_layer)
        net = layers.Dense(64, activation='relu')(net)
        net = layers.Dense(32, activation='relu')(net)
        net = layers.Dense(16, activation='relu')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='softmax')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7649039626121521 | 0.7649039626121521 | 0.7649039626121521 |
| pearson_corr | 0.4780537486076355 | 0.4780537486076355 | 0.4780537486076355 |
| jensen_shannon_divergence | 8.77587890625 | 8.77587890625 | 8.77587890625 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7835187911987305 | 0.7835187911987305 | 0.7835187911987305 |
| pearson_corr | 0.1345556676387787 | 0.1345556676387787 | 0.1345556676387787 |
| jensen_shannon_divergence | 3.5534567832946777 | 3.5534567832946777 | 3.5534567832946777 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.9141563177108765 | 0.9141563177108765 | 0.9141563177108765 |
| pearson_corr | -0.0014357281615957618 | -0.0014357281615957618 | -0.0014357281615957618 |
| jensen_shannon_divergence | 6.227056503295898 | 6.227056503295898 | 6.227056503295898 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.9667567610740662 | 0.9667567610740662 | 0.9667567610740662 |
| pearson_corr | 0.011751626618206501 | 0.011751626618206501 | 0.011751626618206501 |
| jensen_shannon_divergence | 7.6556830406188965 | 7.6556830406188965 | 7.6556830406188965 |


----------------------------



### 0.01, relu, sigmoid

In [33]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='relu')(in_layer)
        net = layers.Dense(64, activation='relu')(net)
        net = layers.Dense(32, activation='relu')(net)
        net = layers.Dense(16, activation='relu')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='sigmoid')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8032684922218323 | 0.8032684922218323 | 0.8032684922218323 |
| pearson_corr | 0.3813827931880951 | 0.3813827931880951 | 0.3813827931880951 |
| jensen_shannon_divergence | 5.960070610046387 | 5.960070610046387 | 5.960070610046387 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6566411256790161 | 0.6566411256790161 | 0.6566411256790161 |
| pearson_corr | 0.46030136942863464 | 0.46030136942863464 | 0.46030136942863464 |
| jensen_shannon_divergence | 2.528578996658325 | 2.528578996658325 | 2.528578996658325 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.9104326963424683 | 0.9104326963424683 | 0.9104326963424683 |
| pearson_corr | -0.047585904598236084 | -0.047585904598236084 | -0.047585904598236084 |
| jensen_shannon_divergence | 5.851060390472412 | 5.851060390472412 | 5.851060390472412 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.9913851022720337 | 0.9913851022720337 | 0.9913851022720337 |
| pearson_corr | 0.0005770056159235537 | 0.0005770056159235537 | 0.0005770056159235537 |
| jensen_shannon_divergence | 16.71843910217285 | 16.71843910217285 | 16.71843910217285 |


----------------------------



### 0.01, relu, relu

In [34]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='relu')(in_layer)
        net = layers.Dense(64, activation='relu')(net)
        net = layers.Dense(32, activation='relu')(net)
        net = layers.Dense(16, activation='relu')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='relu')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 96, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7470260262489319 | 0.7470260262489319 | 0.7470260262489319 |
| pearson_corr | 0.4247940182685852 | 0.4247940182685852 | 0.4247940182685852 |
| jensen_shannon_divergence | 9.311294555664062 | 9.311294555664062 | 9.311294555664062 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5268705487251282 | 0.5268705487251282 | 0.5268705487251282 |
| pearson_corr | 0.5651369094848633 | 0.5651369094848633 | 0.5651369094848633 |
| jensen_shannon_divergence | 1.377767562866211 | 1.377767562866211 | 1.377767562866211 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8449037671089172 | 0.8449037671089172 | 0.8449037671089172 |
| pearson_corr | -0.24526400864124298 | -0.24526400864124298 | -0.24526400864124298 |
| jensen_shannon_divergence | 3.979748249053955 | 3.979748249053955 | 3.979748249053955 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4933841824531555 | 0.4933841824531555 | 0.4933841824531555 |
| pearson_corr | 0.6314826607704163 | 0.6314826607704163 | 0.6314826607704163 |
| jensen_shannon_divergence | 1.2306410074234009 | 1.2306410074234009 | 1.2306410074234009 |


----------------------------

